CPython 인터프리터는 프로그램을 실행할 때 먼저 프로그램을 바이트 코드(bytecode)명령어 시퀸스로 변혼한다. 바이트 코드는 파이썬 가상 시스템이 사용하는 중간 언어로, 성능 최적화를 위해 사용한다. 사람이 읽을 수 있는 소스 코드를 직접 실해하는 대신 컴파일러 구문 및 의미 순석 결과를 나타내는 간단한 숫자 코드, 상수, 참조가 사용된다.  
이렇게 하면 프로그램이나 프로그램 일부를 반복적으로 실행하는 데 드는 시간과 메모리를 절약할 수 있다. 예를 들어 컴퍼일 단계에서 생성된 바이트코드는 .pyc 및 .pyo파일로 디스크에 캐시되므로 두 번째부터는 같은 파이썬 파일을 더 빠르게 실행할 수 있다.  
이 모든 것은 프로그래머에게 완전히 투명하다. 중간 번역 단계나 파이선 가상 머신이 바이트 코드를 처리하는 방법을 알 필요는 없다. 사실 바이트코드 형식은 구현 세부 사항으로 간주되며 파이썬 버전 간 호환성을 보장하지 않는다.  
그런데 Cpython 인터프리터가 제공하는 추상화 뒤에서 실제 코드가 어떻게 만들어지는지 살펴보는 것도 때론 도움이 된다는 사실을 알았다. 내부 동작 중 적어도 일부를 이해하면 성능을 쥐어짜야 하는 중요한 코드를 작성하는 데 도움이 된다. 그리고 그 과정은 무척 재미있다. 
파이썬의 바이트 코드를 이해하기 위해 가지고 놀 수 있는 실험용 샘프로 간단한 greet() 함수를 살펴보자.

In [1]:
def greet(name):
    return 'Hello, ' + name + '!'
greet('Guido')

'Hello, Guido!'

CPython이 소스 코드를 '실행'하기 전에 먼저 중간 언어로 변환한다고 말한 것을 기억하자. 글쎄, 그것이 사실이라면 이 컴파일 단계의 결과를 볼 수 있어야 한다. 물론 볼 수 있다.  
각 함수는 __ code__ 속성을 가지고 있다. 이 속성을 통해 greet() 함수에서 사용하는 가상 ㅓㅁ신 명령어, 상수, 변수를 가져올 수 있다.

In [2]:
greet.__code__.co_code

b'd\x01|\x00\x17\x00d\x02\x17\x00S\x00'

In [3]:
greet.__code__.co_consts

(None, 'Hello, ', '!')

In [4]:
greet.__code__.co_varnames

('name',)

In [5]:
# co_code의 어셈블리어로 되어있어 알아보기 힘듬. 
# dis 모듈로 디스어셈플러 도구를 이용해 쉽게 검사하자.
import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 BINARY_ADD
              6 LOAD_CONST               2 ('!')
              8 BINARY_ADD
             10 RETURN_VALUE


인덱스 1('Hello, ')에서 상수를 검색하여 스택에 저장한다. 그런 다음 name 변수의 내용을 로드하고 스택에 넣는다.  
'스택'은 가상 머신의 내부 작업 저장소로 사용되는 데이터 구조다. 가상 머신에는 여러 가지 종류가 있으며 그 중 하나가 '스택 머신'이다. CPython의 가상 머신은 이러한 스택머신을 구현한 것이다. 앞 코드의 모든 명령어 앞에 '스택'을 붙여보면 이 데이터 구조의 중심 역할이 무엇인지 예상할 수 있다.  
추상적인 데이터 구조로서 스택이 흥미로운 점은 근본적으로 두 가지 작업만 지원한다는 것이다. 바로 푸시와 팝이다.  
스택이 비어있다고 가정해보면 처음 두 연산 코드가 실행된 후 가상 머신 스택의 내용은 다음과 같아진다.  

0: 'Guido' (contents of "name")  
1: 'Hello, '  

BINARY_ADD 명령은 두 문자열 값을 스택에서 팝하고 연결한 다음 결과를 스택에 다시 푸시한다.  

0: 'Hello, Guido' 

그런다음 또 다른 LOAD_CONST가 스택에 느낌표 문자열을 가져온다.

0: '!'  
1: 'Hello, Guido'

다음 BIANRY_ADD 연산코드는 두 가지를 다시 연결하여 최종 인사말 문자열을 생성한다. 

0: 'Hello, Guido!'

마지막 바이트 코드 명령어는 RETURN_VALUE 이며 현재 스택 맨 위에 있는 것이 이 함수의 반환값이므로 호출자에게 전달될 수 있도록 가상 머신에 알린다.  
이 주제에 대한 책으로 빌헬름(Wilhelm)과 사이들(Seidl)이 쓴 "Compiler Design : Virtual Machines"를 추천한다.